In [1]:
"""Markowitz investment strategy based on a moving average model"""

import pandas as pd
import numpy as np

file_name = '收益率.xlsx'
sheet_name = '涨跌幅' # 数据单位是 %
stocks = ['伊利股份','美的集团','招商银行','中信证券','恒瑞医药','华大基因','哔哩哔哩','宁德时代','科大讯飞','新东方']
"""stocks代码可优化"""

n = len(stocks)

df = pd.read_excel(file_name, sheet_name=sheet_name)

def read_stock_data(file_name, sheet_name, stocks):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    stock_data_list = []
    for stock in stocks:
        if stock in df.columns:
            stock_data = df[stock].values
            stock_data_list.append(stock_data)
        else:  
            print(f"股票 {stock} 不在数据框中，请检查股票名称是否正确或数据是否完整。")
    data_array = np.vstack(stock_data_list).T
    data_array /= 100 #去百分号
    return data_array

data_array = read_stock_data(file_name, sheet_name, stocks)
print(np.shape(data_array))
print(data_array)

(469, 10)
[[-0.011905   -0.034362   -0.02713    ... -0.072727   -0.050591
  -0.04343105]
 [ 0.025465    0.037572   -0.004357   ...  0.028322   -0.004695
   0.00908059]
 [ 0.006142   -0.015146    0.001094   ... -0.012712   -0.016274
  -0.01687289]
 ...
 [-0.014456   -0.02113     0.005275   ... -0.015431   -0.013848
   0.02023988]
 [ 0.0033     -0.004287   -0.002778   ...  0.007194    0.003404
  -0.02645114]
 [-0.006213    0.001999    0.005571   ... -0.016633   -0.026718
   0.02037736]]


In [2]:
"""上一天到下一天视为时刻0和时刻1"""
"""计算从第6天开始，每一天10只股票的期望收益率 E_r = expected_returns """
expected_returns = np.zeros((464, n))

for date in range(5, 469):
    past_5_days_data = data_array[date-5:date, :]
    expected_returns[date-5, :] = past_5_days_data.mean(axis=0)

print(expected_returns.shape)
print(expected_returns)

(464, 10)
[[ 0.0084152   0.004276   -0.0119808  ... -0.0188472  -0.013115
  -0.02380368]
 [ 0.014361    0.012056   -0.0113054  ... -0.019411   -0.0031938
  -0.01266348]
 [ 0.0072078  -0.0007746  -0.009558   ... -0.0240386  -0.0069886
  -0.01860081]
 ...
 [-0.0074598   0.006855    0.0028208  ...  0.0042706  -0.004652
  -0.01582088]
 [-0.0098576   0.0032014  -0.0028228  ... -0.0063914  -0.0029826
  -0.00467392]
 [-0.0079966  -0.0006854  -0.0027612  ... -0.0009902   0.0067362
  -0.00458552]]


In [3]:
"""计算从第6天开始，每一天10只股票的收益率协方差矩阵 sigma_r = covariance_returns"""
daily_covariance = []

for i in range(5, 469):
    past_5_days_data = data_array[i-5:i, :]
    covariance_5 = np.cov(past_5_days_data, rowvar=False)
    daily_covariance.append(covariance_5)

covariance_returns = np.array(daily_covariance)
print(covariance_returns.shape)
print(covariance_returns)

(464, 10, 10)
[[[ 3.07169694e-04  5.35938385e-04  2.17098886e-04 ...  5.13409136e-04
    2.19007589e-04  4.54799424e-04]
  [ 5.35938385e-04  1.04920870e-03  2.88969162e-04 ...  9.41166569e-04
    5.04969248e-04  6.80146759e-04]
  [ 2.17098886e-04  2.88969162e-04  2.30610051e-04 ...  3.52957846e-04
    9.57225220e-05  3.93201545e-04]
  ...
  [ 5.13409136e-04  9.41166569e-04  3.52957846e-04 ...  1.30524801e-03
    5.91181223e-04  7.38372605e-04]
  [ 2.19007589e-04  5.04969248e-04  9.57225220e-05 ...  5.91181223e-04
    5.08864518e-04  1.08412653e-04]
  [ 4.54799424e-04  6.80146759e-04  3.93201545e-04 ...  7.38372605e-04
    1.08412653e-04  8.59460652e-04]]

 [[ 1.81882769e-04  2.82448785e-04  1.07429882e-04 ...  1.10520443e-04
   -1.65770170e-05  3.57147019e-04]
  [ 2.82448785e-04  6.00341600e-04  1.35296121e-04 ...  4.22483825e-04
    4.72806710e-05  3.84580548e-04]
  [ 1.07429882e-04  1.35296121e-04  2.07311452e-04 ...  3.16242271e-04
   -9.02855024e-05  2.03295219e-04]
  ...
  [ 1.105

In [4]:
def calculate_wd(expected_returns, covariance_returns):
    wds = []

    # 遍历每一天的期望收益率和协方差矩阵
    for i in range(expected_returns.shape[0]):
        Er = expected_returns[i]
        sigma = covariance_returns[i]
        e = np.ones(n)

        # 计算wd = sigma^(-1) * Er / (e^T * sigma^(-1) * Er)
        '''计算sigma逆矩阵的方法可优化'''
        sigma_inv = np.linalg.inv(sigma)
        wd_numerator = np.dot(sigma_inv, Er)
        wd_denominator = np.dot(e.T, np.dot(sigma_inv, Er))
        wd = wd_numerator / wd_denominator
        
        wds.append(wd)  

    # 将列表转换为NumPy数组
    wd_array = np.array(wds)

    return wd_array  

wd_array = calculate_wd(expected_returns, covariance_returns)
print(wd_array.shape)
print(wd_array)

(464, 10)
[[-6.29481418e-01 -4.09073308e-01  1.80249821e+00 ...  3.30282247e-02
  -3.80799718e-01 -7.27260115e-01]
 [-1.01182750e+00 -1.02236784e+00  4.85597090e-01 ... -1.98163900e-01
   1.51692862e+00  1.31527288e+00]
 [ 3.27685216e-01 -3.08803217e-01  5.06669617e-02 ...  8.76143940e-02
   5.71944841e-01  1.02447627e-01]
 ...
 [-8.99322779e-01  1.74054520e+00  2.52541633e-01 ...  4.73260627e-01
   4.46231879e-02 -2.10854323e-02]
 [ 9.91523663e-01  1.91013777e-02  7.81937364e-01 ...  1.23265859e-01
   8.16490260e-02  1.29033028e-01]
 [ 2.74322859e-01  2.82615319e-01  2.78911938e-01 ...  1.80043804e-01
   1.13613104e-04  2.32288308e-01]]


In [5]:
'''投资权重标准化'''
def wd_scale(wd_array):
    row_sums = wd_array.sum(axis=1)
    scale_factors = 100 / row_sums # 计算缩放系数，即100除以每行的和
    scale_factors[np.isinf(scale_factors)] = 1 # 如果出现除以零的情况，将缩放系数设为1
    scaled_wd = wd_array * scale_factors[:, np.newaxis]
    # 将每一行按原比例缩放，使得每行的和为100
    # 使用[:, np.newaxis]将缩放系数扩展为列向量，以便进行广播
    return scaled_wd

scaled_wd = wd_scale(wd_array)

In [6]:
'''反推本金（假定先观望5天，从第6天最低价时买入，连续投资464个交易日后平仓）'''
# 读取第6天最低价
file_name = '本金.xlsx'
sheet_name = '前六天最低价'
df = pd.read_excel(file_name, sheet_name=sheet_name)

def read_min(file_name, sheet_name, stocks):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    min_list = []
    for stock in stocks:
        if stock in df.columns:
            min_data = df[stock].values
            min_list.append(min_data)
        else:  
            print(f"股票 {stock} 不在数据框中，请检查股票名称是否正确或数据是否完整。")
    min_list = np.vstack(min_list).T
    
    return min_list

min_list = read_min(file_name, sheet_name, stocks)
# print(min_list.shape)
# print(min_list)

min_6 = min_list[5]
print(min_6)
scaled_wd_6 = scaled_wd[5]

principal_each_stock = min_6 * scaled_wd_6
principal = sum(principal_each_stock)
print(principal)

[ 35.7606294   52.60908024  37.14159389  18.93454563  32.96460666
  71.21810405 181.3        223.62343866  40.24277897   8.23      ]
14144.580373930145


In [7]:
file_name = '净收益.xlsx'
sheet_name = '净收益率'
df = pd.read_excel(file_name, sheet_name=sheet_name)

def read_interest(file_name, sheet_name, stocks):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    stock_interest_list = []
    for stock in stocks:
        if stock in df.columns:
            stock_interest = df[stock].values
            stock_interest_list.append(stock_interest)
        else:
            print(f"股票 {stock} 不在数据框中，请检查股票名称是否正确或数据是否完整。")
    
    interest_array = np.vstack(stock_interest_list).T
    
    return interest_array

interest_array = read_interest(file_name, sheet_name, stocks)
interest_array = interest_array[5:,:] # 删除最开始5天的数据
print(np.shape(interest_array))
print(interest_array)

(464, 10)
[[ 0.01651376 -0.00331588 -0.01662708 ... -0.07270704 -0.00837438
  -0.01717791]
 [-0.00386299 -0.0185893   0.01411192 ... -0.0068059  -0.01060158
  -0.01454545]
 [ 0.00884725  0.02445119  0.04457364 ...  0.01794138 -0.01111111
   0.02970297]
 ...
 [-0.00831225 -0.02098007  0.00496432 ... -0.01543132 -0.01405791
   0.01949025]
 [ 0.00366703  0.00061237 -0.0058642  ...  0.01027749  0.00340426
  -0.01102131]
 [-0.00694444  0.00461255  0.00928505 ... -0.0069898  -0.02671756
   0.02037736]]


In [17]:
"""计算投资净收益和收益波动"""
def calculate_profit(scaled_wd, interest_array):
    profit = scaled_wd * interest_array
    profit_result = profit.sum(axis=1)
    profit_sum = sum(profit_result)
    profit_deviation = np.var(profit_result)
#     print(profit_sum)
#     print(profit.shape)
#     print(profit)
#     print(profit_result.shape)
#     print(profit_result)
#     print(profit_sum.shape)
    return profit_sum, profit_deviation

In [21]:
'''波动'''
file_name = '净收益.xlsx'
sheet_arbitrage = '最大套利空间'
df = pd.read_excel(file_name, sheet_name=sheet_arbitrage)

arbitrage_array = read_interest(file_name, sheet_name, stocks) # 再次调用这个函数用于读取最大套利空间数据
arbitrage_array = arbitrage_array[5:,:] # 删除最开始5天的数据

def var_arbtrg(scaled_wd,arbitrage_array):
    wd_squared = scaled_wd * scaled_wd
    arbtrg = wd_squared * arbitrage_array
    arbtrg_result = arbtrg.sum(axis=1)
    arbtrg_variance = np.var(arbtrg_result)
    return arbtrg_variance

In [22]:
"""该种策略"""
calculate_profit(scaled_wd, interest_array)

(-22.860637565856866, 9.05900715511032)

In [23]:
var_arbtrg(scaled_wd,arbitrage_array)

23811943.10394687

In [10]:
"""等权重投资策略"""
equal_scaled_wd = (100/n) * np.ones(n) # 已经经过标准化scaled
calculate_profit(equal_scaled_wd, interest_array)

(37.09437279766784, 1.7925468814195413)

In [24]:
var_arbtrg(equal_scaled_wd,arbitrage_array)

122.43336393822433

In [11]:
"""全局最小方差投资策略"""
def calculate_mvp_wd(covariance_returns):
    mvp_wds = []

    # 遍历每一天的期望收益率和协方差矩阵
    for i in range(expected_returns.shape[0]):
        sigma = covariance_returns[i]
        e = np.ones(10)

        # 计算wd = sigma^(-1) * e / (e^T * sigma^(-1) * e)
        '''计算sigma逆矩阵的方法可优化'''
        sigma_inv = np.linalg.inv(sigma)
        mvp_wd_numerator = np.dot(sigma_inv, e)
        mvp_wd_denominator = np.dot(e.T, np.dot(sigma_inv, e))
        mvp_wd = mvp_wd_numerator / mvp_wd_denominator
        
        mvp_wds.append(mvp_wd)  

    # 将列表转换为NumPy数组
    mvp_wd_array = np.array(mvp_wds)

    return mvp_wd_array  

mvp_wd_array = calculate_mvp_wd(covariance_returns)
mvp_scaled_wd = wd_scale(mvp_wd_array)
calculate_profit(mvp_scaled_wd, interest_array)

(-10504.315300234197, 213306.819090507)

In [25]:
var_arbtrg(mvp_scaled_wd,arbitrage_array)

7.218067913161192e+16

In [27]:
'''加入无风险债券，重来一遍上述代码（待优化）'''
file_name = '收益率.xlsx'
sheet_name = '涨跌幅'
stocks = ['伊利股份','美的集团','招商银行','中信证券','恒瑞医药','华大基因','哔哩哔哩','宁德时代','科大讯飞','新东方']
n = len(stocks)
rf = 0.01 # 无风险债券的收益率为 1 %
w0 = 5 # 投资于无风险债券的比例为 5 %

df = pd.read_excel(file_name, sheet_name=sheet_name)

def read_stock_data(file_name, sheet_name, stocks):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    stock_data_list = []
    for stock in stocks:
        if stock in df.columns:
            stock_data = df[stock].values
            stock_data_list.append(stock_data)
        else:  
            print(f"股票 {stock} 不在数据框中，请检查股票名称是否正确或数据是否完整。")
    data_array = np.vstack(stock_data_list).T
    data_array /= 100
    return data_array

data_array = read_stock_data(file_name, sheet_name, stocks)

expected_returns = np.zeros((464, n))

for date in range(5, 469):
    past_5_days_data = data_array[date-5:date, :]
    expected_returns[date-5, :] = past_5_days_data.mean(axis=0)

daily_covariance = []

for i in range(5, 469):
    past_5_days_data = data_array[i-5:i, :]
    covariance_5 = np.cov(past_5_days_data, rowvar=False)
    daily_covariance.append(covariance_5)

covariance_returns = np.array(daily_covariance)

def calculate_wd(expected_returns, covariance_returns):
    wds = []

    # 遍历每一天的期望收益率和协方差矩阵
    for i in range(expected_returns.shape[0]):
        Er = expected_returns[i]
        sigma = covariance_returns[i]
        e = np.ones(n)

        # 计算wd = sigma^(-1) * Er / (e^T * sigma^(-1) * Er)
        '''计算sigma逆矩阵的方法可优化'''
        sigma_inv = np.linalg.inv(sigma)
        C = np.dot(e.T, sigma_inv, e)
        A = np.dot(e.T, np.dot(sigma_inv, Er))
        wd_denominator = A - C * rf
        wd_numerator = np.dot(sigma_inv, Er - rf * e)
        wd = wd_numerator / wd_denominator
        wds.append(wd)
    
    wd_array = np.array(wds)

    return wd_array

wd_array = calculate_wd(expected_returns, covariance_returns)

def wd_scale(wd_array):
    
    row_sums = wd_array.sum(axis=1)
    scale_factors = (100 - w0) / row_sums # 计算缩放系数，即100除以每行的和
    scale_factors[np.isinf(scale_factors)] = 1 # 如果出现除以零的情况，将缩放系数设为1
    scaled_wd = wd_array * scale_factors[:, np.newaxis]
    return scaled_wd

scaled_wd = wd_scale(wd_array)

file_name = '本金.xlsx'
sheet_name = '前六天最低价'
df = pd.read_excel(file_name, sheet_name=sheet_name)

def read_min(file_name, sheet_name, stocks):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    min_list = []
    for stock in stocks:
        if stock in df.columns:
            min_data = df[stock].values
            min_list.append(min_data)
        else:  
            print(f"股票 {stock} 不在数据框中，请检查股票名称是否正确或数据是否完整。")
    min_list = np.vstack(min_list).T
    
    return min_list

min_list = read_min(file_name, sheet_name, stocks)
min_6 = min_list[5]
scaled_wd_6 = scaled_wd[5]

principal_each_stock = min_6 * scaled_wd_6
principal = sum(principal_each_stock)
print('本金：',principal)

file_name = '净收益.xlsx'
sheet_name = '净收益率'
df = pd.read_excel(file_name, sheet_name=sheet_name)

def read_interest(file_name, sheet_name, stocks):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    stock_interest_list = []
    for stock in stocks:
        if stock in df.columns:
            stock_interest = df[stock].values
            stock_interest_list.append(stock_interest)
        else:
            print(f"股票 {stock} 不在数据框中，请检查股票名称是否正确或数据是否完整。")
    interest_array = np.vstack(stock_interest_list).T
#     print(interest_array)
    return interest_array

interest_array = read_interest(file_name, sheet_name, stocks)
interest_array = interest_array[5:,:] # 删除最开始5天的数据

def calculate_profit(scaled_wd, interest_array):
    profit = scaled_wd * interest_array
    profit_result = profit.sum(axis=1)
    profit_sum = sum(profit_result)
    return profit_sum

'''波动'''
file_name = '净收益.xlsx'
sheet_arbitrage = '最大套利空间'
df = pd.read_excel(file_name, sheet_name=sheet_arbitrage)

arbitrage_array = read_interest(file_name, sheet_name, stocks) # 再次调用这个函数用于读取最大套利空间数据
arbitrage_array = arbitrage_array[5:,:] # 删除最开始5天的数据

def var_arbtrg(scaled_wd,arbitrage_array):
    wd_squared = scaled_wd * scaled_wd
    arbtrg = wd_squared * arbitrage_array
    arbtrg_result = arbtrg.sum(axis=1)
    arbtrg_variance = np.var(arbtrg_result)
    return arbtrg_variance


"""该种策略"""
profit_1 = calculate_profit(scaled_wd, interest_array) + w0 * rf
arbtrg_variance_1 = var_arbtrg(scaled_wd,arbitrage_array)
print('1：',profit_1,arbtrg_variance_1)

"""等权重投资策略"""
w0 = 100/(n+1)
equal_scaled_wd = (100/(n+1)) * np.ones(n) # 已经经过标准化scaled
profit_2 = calculate_profit(equal_scaled_wd, interest_array) + w0 * rf
arbtrg_variance_2 = var_arbtrg(equal_scaled_wd,arbitrage_array)
print('2：',profit_2,arbtrg_variance_2)

"""全局最小方差投资策略，净收益就是 (principal)*(rf)"""
profit_3 = principal * (rf)
arbtrg_variance_3 = var_arbtrg(mvp_scaled_wd,arbitrage_array)
print('3：',profit_3,arbtrg_variance_3)

本金： 13005.026520650987
1： -22.810637565856865 23811943.10394687
2： 33.813066179698005 122.43336393822433
3： 130.05026520650986 7.218067913161192e+16
